## Используем сервис MeaningCloud

Сервис [MeaningCloud.com](http://MeaningCloud.com) предоставляет 20000 запросов в месяц, и содержит продвинутые средства анализа естественного языка на нескольких языках.

Вам необходимо войти в сервис с помощью GitHub или зарегистрировшись на нём, после чего вы сразу получите ключ для вызова API.

Попробуем использовать сервис для извлечения ключевых слов и анализа предложений (topic modeling):

In [2]:
import requests
import sys

key = '=== YOUR KEY HERE ==='

def analyze(x,lang='en',topics='a'):
    url = "https://api.meaningcloud.com/topics-2.0"

    payload={
        'key': key,
        'txt': x,
        'lang': lang,  # 2-letter code, like en es fr ...
        'tt': topics
    }

    response = requests.post(url, data=payload)
    return response.json()

#analyze('I would love to visit Torino one day; it seems like one of the best cities in Italy!')
analyze('I would love to visit Torino one day, even though I have been there two times!')

{'concept_list': [],
 'entity_list': [{'form': 'Turin',
   'id': '0729ecde4a',
   'relevance': '100',
   'sementity': {'class': 'instance',
    'fiction': 'nonfiction',
    'id': 'ODENTITY_CITY',
    'type': 'Top>Location>GeoPoliticalEntity>City'},
   'semgeo_list': [{'adm1': {'form': 'Piemonte', 'id': '25836717d4'},
     'adm2': {'form': 'Turin', 'id': '4ccf5970b7'},
     'continent': {'form': 'Europe', 'id': '0404ea4d6c'},
     'country': {'form': 'Italy',
      'id': '85c9d18db8',
      'standard_list': [{'id': 'ISO3166-1-a2', 'value': 'IT'},
       {'id': 'ISO3166-1-a3', 'value': 'ITA'}]}}],
   'semld_list': ['http://en.wikipedia.org/wiki/Turin',
    'http://ar.wikipedia.org/wiki/تورينو',
    'http://ca.wikipedia.org/wiki/Torí',
    'http://cs.wikipedia.org/wiki/Turín',
    'http://da.wikipedia.org/wiki/Torino',
    'http://de.wikipedia.org/wiki/Turin',
    'http://es.wikipedia.org/wiki/Turín',
    'http://fi.wikipedia.org/wiki/Torino',
    'http://fr.wikipedia.org/wiki/Turin',
   

Интересно, что сервис осуществляет привязку объектов к некоторой [внутренней онтологии](https://www.meaningcloud.com/developer/documentation/ontology#ODTHEME_TOP), а также к внешним ссылкам на Wikipedia.

Применим этот сервис к роману "Анна Каренина". Для начала разобьем текст на абзацы:

In [7]:
text = open('d:/git/ai_miit/data/akar_en.txt',encoding='utf-8').read()
novel = text[2:].split('\n')[61:]

def bypar(text):
    s = ""
    for x in text:
        if x == "":
            if s!="":
                yield s[:-1]
            s=""
        else:
            s+=x+" "

list(bypar(novel))[:5]

['Happy families are all alike; every unhappy family is unhappy in its own way.',
 'Everything was in confusion in the Oblonskys’ house. The wife had discovered that the husband was carrying on an intrigue with a French girl, who had been a governess in their family, and she had announced to her husband that she could not go on living in the same house with him. This position of affairs had now lasted three days, and not only the husband and wife themselves, but all the members of their family and household, were painfully conscious of it. Every person in the house felt that there was no sense in their living together, and that the stray people brought together by chance in any inn had more in common with one another than they, the members of the family and household of the Oblonskys. The wife did not leave her own room, the husband had not been at home for three days. The children ran wild all over the house; the English governess quarreled with the housekeeper, and wrote to a friend 

Выделим первые 5 абзацев:

In [8]:
import itertools as itt
text = "\n".join(itt.islice(bypar(novel),5))
print(text)

Happy families are all alike; every unhappy family is unhappy in its own way.
Everything was in confusion in the Oblonskys’ house. The wife had discovered that the husband was carrying on an intrigue with a French girl, who had been a governess in their family, and she had announced to her husband that she could not go on living in the same house with him. This position of affairs had now lasted three days, and not only the husband and wife themselves, but all the members of their family and household, were painfully conscious of it. Every person in the house felt that there was no sense in their living together, and that the stray people brought together by chance in any inn had more in common with one another than they, the members of the family and household of the Oblonskys. The wife did not leave her own room, the husband had not been at home for three days. The children ran wild all over the house; the English governess quarreled with the housekeeper, and wrote to a friend asking

Проанализируем их как единый текст:

In [12]:
res = analyze(text)

Результат запишем в файл и посмотрим:

In [13]:
import json
with open('res.json','w',encoding='utf-8') as f:
    json.dump(res,f)

Видно, что текста анализируется целиком как единое целое - это удобно для выделения сущностей, но не удобно для анализа отдельных предложений. Поэтому вызовем сервис отдельно для предложения:

In [32]:
text = next(itt.islice(bypar(novel),1,2))
lines = text.split('. ')
lines

['Everything was in confusion in the Oblonskys’ house',
 'The wife had discovered that the husband was carrying on an intrigue with a French girl, who had been a governess in their family, and she had announced to her husband that she could not go on living in the same house with him',
 'This position of affairs had now lasted three days, and not only the husband and wife themselves, but all the members of their family and household, were painfully conscious of it',
 'Every person in the house felt that there was no sense in their living together, and that the stray people brought together by chance in any inn had more in common with one another than they, the members of the family and household of the Oblonskys',
 'The wife did not leave her own room, the husband had not been at home for three days',
 'The children ran wild all over the house; the English governess quarreled with the housekeeper, and wrote to a friend asking her to look out for a new situation for her; the man-cook ha

Проанализируем одно предложение. В этом примере: *The wife had discovered that the husband was carrying on an intrigue with a French girl, who had been a governess in their family, and she had announced to her husband that she could not go on living in the same house with him*

In [35]:
res = analyze(lines[1])

In [36]:
res

{'concept_list': [{'form': 'husband',
   'id': '6a17861b62',
   'relevance': '100',
   'sementity': {'class': 'class',
    'fiction': 'nonfiction',
    'id': 'ODENTITY_PERSON',
    'type': 'Top>Person'},
   'semld_list': ['http://en.wikipedia.org/wiki/Husband',
    'http://ar.wikipedia.org/wiki/زوج',
    'http://ca.wikipedia.org/wiki/Marit',
    'http://he.wikipedia.org/wiki/בעל_(משפחה)',
    'http://id.wikipedia.org/wiki/Suami',
    'http://it.wikipedia.org/wiki/Marito',
    'http://ko.wikipedia.org/wiki/남편',
    'http://no.wikipedia.org/wiki/Ektemann',
    'http://ro.wikipedia.org/wiki/Soț',
    'http://ru.wikipedia.org/wiki/Муж',
    'http://sv.wikipedia.org/wiki/Make_(äktenskap)',
    'http://zh.wikipedia.org/wiki/丈夫',
    'http://d-nb.info/gnd/4070668-0',
    'sumo:Human'],
   'variant_list': [{'endp': '39', 'form': 'husband', 'inip': '33'},
    {'endp': '166', 'form': 'husband', 'inip': '160'}]}],
 'entity_list': [],
 'money_expression_list': [],
 'other_expression_list': [],
 'q

Выделим из этого предложения отдельные связи, отвечающие за атомарные фрагменты смысла:

In [14]:
def complist(l):
    return ', '.join([x['form'] for x in l])

for r in res['relation_list']:
    print(f"{r['subject']['lemma_list']} -> {r['verb']['lemma_list']} -> {complist(r['complement_list'])}")

['family'] -> ['be'] -> all alike, in confusion, in the Oblonskys, unhappy in its own way
['family'] -> ['house'] -> 
['wife'] -> ['discover'] -> that the husband was carrying on an intrigue with a French girl
['husband'] -> ['be'] -> a governess, in their family, was carrying, who had been a governess in their family, at home for three days
['husband'] -> ['carry'] -> on an intrigue with a French girl
['she'] -> ['be'] -> sense
['she'] -> ['yes'] -> 
['she'] -> ['there was'] -> no sense, in their living, together
['she'] -> ['announce'] -> to her husband
['she'] -> ['go on'] -> 
['position'] -> ['last'] -> now, three days
['member'] -> ['be'] -> position
['person'] -> ['feel'] -> that there was no sense in their living together
['people'] -> ['bring'] -> together by chance, in any inn had more in common with one another than they
['wife'] -> ['leave'] -> her own room
['child'] -> ['run'] -> wild, all over
['governess'] -> ['write'] -> to a friend
['governess'] -> ['quarrel'] -> with t